In [97]:
!pip install tabulate

In [98]:
from tabulate import tabulate

## Part : 2

In [99]:
# grammar rules

grammar_rules = {'S': [['NP', 'VP']],
         'NP':[['JJ', 'NP'], ['British'], ['waffles'], ['Falklands'], ['left']],
         'VP': [['VP', 'NP'], ['VP', 'PP'], ['left'], ['waffles']],
         'PP' : [['P', 'NP']],
         'P' : [['on']],
        'JJ': [['British']]}

In [100]:
def CKY_parser(tokens):
    print(tokens)
    token_len = len(tokens)
    
    # set up table
    
    table = [[set([]) for j in range(token_len)] for i in range(token_len)]
    
    for j in range(0, token_len):
        for pos, rule in grammar_rules.items():
            for rule_token in rule:
                if len(rule_token) == 1 and rule_token[0] == tokens[j]:
                    table[j][j].add(pos)

        for i in range(j, -1, -1):
            for k in range(i, j):
                for pos, rule in grammar_rules.items():
                    for rule_token in rule:
                        if len(rule_token) == 2 and rule_token[0] in table[i][k] and rule_token[1] in table[k+1][j]:
                            table[i][j].add(pos)
               
    return table

In [101]:
sent = 'British left waffles on Falklands'

words = sent.split(' ')
parse_table = CKY_parser(words)

print(tabulate(parse_table))

['British', 'left', 'waffles', 'on', 'Falklands']
------------  ------------  ------------  -----  -----------
{'JJ', 'NP'}  {'S', 'NP'}   {'S'}         set()  {'S'}
set()         {'VP', 'NP'}  {'S', 'VP'}   set()  {'S', 'VP'}
set()         set()         {'VP', 'NP'}  set()  {'VP'}
set()         set()         set()         {'P'}  {'PP'}
set()         set()         set()         set()  {'NP'}
------------  ------------  ------------  -----  -----------


## Part : 3

In [102]:
grammar_rules = {'S': [(['NP', 'VP'],1.0)],
         'NP':[(['NP', 'PP'], 0.4), (['astronomers'],0.4), (['ears'],0.18), (['saw'],0.04), (['stars'], 0.18), (['telescopes'],0.1)],
         'VP': [(['V', 'NP'], 0.7), (['VP', 'PP'], 0.3)],
         'PP' : [(['P', 'NP'], 1.0)],
         'P' : [(['with'], 1.0)],
         'V' : [(['saw'], 1.0)]}

In [103]:
def PCKY_parser(tokens):
    print(tokens)
    token_len = len(tokens)
    
    # set up table
    
    table = [[set([]) for j in range(token_len)] for i in range(token_len)]
    
    for j in range(0, token_len):
        for pos, rule in grammar_rules.items():
            for rule_token in rule:
                if len(rule_token[0]) == 1 and rule_token[0][0] == tokens[j]:
                    if len(table[j][j]) == 0:
                        table[j][j].add((pos,rule_token[1]))
                    else:
                        if rule_token[1] > list(table[0][0])[0][1]:
                            table[j][j] = set([])
                            table[j][j].add((pos,rule_token[1]))

        for i in range(j, -1, -1):
            for k in range(i, j):
                for pos, rule in grammar_rules.items():
                    for rule_token in rule:
                        if len(rule_token[0]) == 2:
                            if len(list(table[i][k])) > 0:
                                i_k_eles = [item for item in list(table[i][k]) if rule_token[0][0] == item[0]]
                                for i_k_ele in i_k_eles:
                                    if len(list(table[k+1][j])) > 0:
                                        k_j_eles = [item for item in list(table[k+1][j]) if rule_token[0][1] == item[0]]
                                        for k_j_ele in k_j_eles:
                                            prob = rule_token[1] * i_k_ele[1] * k_j_ele[1]
                                            if len(table[i][j]) == 0:
                                                table[i][j].add((pos,round(prob, 5)))
                                            else:
                                                if prob > list(table[i][j])[0][1]:
                                                    table[i][j].add((pos,round(prob, 5)))
                                               
    return table

In [104]:
sent = 'astronomers saw stars with ears'

words = sent.split(' ')
parse_table = PCKY_parser(words)
print(tabulate(parse_table))

['astronomers', 'saw', 'stars', 'with', 'ears']
-------------  ------------  ---------------  ------------  -----------------
{('NP', 0.4)}  set()         {('S', 0.0504)}  set()         {('S', 0.00363)}
set()          {('V', 1.0)}  {('VP', 0.126)}  set()         {('VP', 0.00907)}
set()          set()         {('NP', 0.18)}   set()         {('NP', 0.01296)}
set()          set()         set()            {('P', 1.0)}  {('PP', 0.18)}
set()          set()         set()            set()         {('NP', 0.18)}
-------------  ------------  ---------------  ------------  -----------------


In [105]:
def generate_tree(table, words):
    parse_tree = ['S']
    rows = len(table)
    for i in range(rows):
        rules = grammar_rules[list(table[i][rows-1])[0][0]]
        if len(rules) == 1:
            lhs = []
            lhs.append(rules[0][0][0])
            lhs.append(words[i])
            rhs = []
            rhs.append(rules[0][0][1])
            parse_tree.append(lhs)
            parse_tree.append(rhs)
        elif len(rules) == 2:
            lhs = []
            rhs = []
            if rules[0][1] > rules[1][1]:
                lhs.append(rules[0][0][0])
                lhs.append(words[i])
                rhs.append(rules[0][0][1])
                parse_tree.append(lhs)
                parse_tree.append(rhs)
        elif list(table[i][rows-1])[0][0] == 'NP':
            lhs = []
            lhs.append(list(table[i][rows-1])[0][0])
            lhs.append(words[i])
            parse_tree.append(lhs)
            
    return parse_tree

In [106]:
parse_tree_string = generate_tree(parse_table, words)
parse_tree_string

['S',
 ['NP', 'astronomers'],
 ['VP'],
 ['V', 'saw'],
 ['NP'],
 ['NP', 'stars'],
 ['P', 'with'],
 ['NP'],
 ['NP', 'ears']]

In [107]:
# probability of parse tree

print(list(parse_table[0][4])[0][1])

0.00363


## Part : 4

In [108]:
def WCKY_parser(tokens):
    print(tokens)
    token_len = len(tokens)
    
    # set up table
    
    table = [[set([]) for j in range(token_len)] for i in range(token_len)]
    
    for j in range(0, token_len):
        for pos, rule in grammar_rules.items():
            for rule_token in rule:
                if len(rule_token[0]) == 1 and rule_token[0][0] == tokens[j]:
                    table[j][j].add((pos,rule_token[1]))

        for i in range(j, -1, -1):
            for k in range(i, j):
                for pos, rule in grammar_rules.items():
                    for rule_token in rule:
                        if len(rule_token[0]) == 2:
                            if len(list(table[i][k])) > 0:
                                i_k_eles = [item for item in list(table[i][k]) if rule_token[0][0] == item[0]]
                                for i_k_ele in i_k_eles:
                                    if len(list(table[k+1][j])) > 0:
                                        k_j_eles = [item for item in list(table[k+1][j]) if rule_token[0][1] == item[0]]
                                        for k_j_ele in k_j_eles:
                                            prob = rule_token[1] * i_k_ele[1] * k_j_ele[1]
                                            table[i][j].add((pos,round(prob, 5)))
           
    return table

In [109]:
sent = 'astronomers saw stars with ears'

words = sent.split(' ')
parse_table = WCKY_parser(words)
print(tabulate(parse_table))

['astronomers', 'saw', 'stars', 'with', 'ears']
-------------  --------------------------  ---------------  ------------  ---------------------------------
{('NP', 0.4)}  set()                       {('S', 0.0504)}  set()         {('S', 0.00363), ('S', 0.00272)}
set()          {('V', 1.0), ('NP', 0.04)}  {('VP', 0.126)}  set()         {('VP', 0.0068), ('VP', 0.00907)}
set()          set()                       {('NP', 0.18)}   set()         {('NP', 0.01296)}
set()          set()                       set()            {('P', 1.0)}  {('PP', 0.18)}
set()          set()                       set()            set()         {('NP', 0.18)}
-------------  --------------------------  ---------------  ------------  ---------------------------------


In [110]:
# probability of the sentence
final_probab = parse_table[0][4]
total_sentence_probab = 0
for item in list(final_probab):
    total_sentence_probab += item[1]
    
print(total_sentence_probab)

0.00635
